In [1]:
from pytorch3d.io import load_obj
import numpy as np
import cv2
verts, faces, aux = load_obj('../test_data/000220.obj')

/home/aaron/miniconda3/envs/mp/lib/python3.10/site-packages/pytorch3d/io/obj_io.py:544: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


In [2]:
rotation=[
    "0.99901461745294595",
    "-0.010370447633126606",
    "-0.043153770764936143",
    "0.0045613799688059235",
    "0.99116007160294672",
    "-0.13259300989426953",
    "0.04414734338693841",
    "0.13226551431091271",
    "0.99023070331915353"
]

In [4]:
rotation = np.array(rotation)

In [6]:
rotation.reshape((3, 3)).T

array([['0.99901461745294595', '0.0045613799688059235',
        '0.04414734338693841'],
       ['-0.010370447633126606', '0.99116007160294672',
        '0.13226551431091271'],
       ['-0.043153770764936143', '-0.13259300989426953',
        '0.99023070331915353']], dtype='<U21')

In [60]:
arr = (aux.verts_uvs*1023).numpy().astype(np.int32)[faces.textures_idx.numpy()]

In [61]:
indices = np.where(np.all(arr == [0, 0], axis=-1))

In [62]:
tri2uv = (aux.verts_uvs*(size-1)).numpy().astype(np.int32)[faces.textures_idx.numpy()]

In [4]:
def point_in_triangle(x, y, p1, p2, p3):
    d1 = cross_product(x, y, p1[0], p1[1], p2[0], p2[1])
    d2 = cross_product(x, y, p2[0], p2[1], p3[0], p3[1])
    d3 = cross_product(x, y, p3[0], p3[1], p1[0], p1[1])
    
    has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0)
    has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0)
    
    return not (has_neg and has_pos)

def cross_product(x1, y1, x2, y2, x3, y3):
    return (x2 - x1) * (y3 - y1) - (x3 - x1) * (y2 - y1)

In [5]:
def get_tri_inner_pixel(vertices):
    
    min_corner = np.min(vertices, axis=0)
    max_corner = np.max(vertices, axis=0)
    inner = []
    for y in range(min_corner[0], max_corner[0]):
        for x in range(min_corner[1], max_corner[1]):
            if point_in_triangle(y,x,vertices[0],vertices[1], vertices[2]):
                inner.append([y,x])
    return inner            

In [6]:
size = (1024, 1024)
uv_color = {}
for i, tri in enumerate(tri2uv):
    inside = get_tri_inner_pixel(tri)
    uv_color[i] = {'triangle': tri, 'inside': np.array(inside)}
    
    
        

In [9]:
size = 1024
image = cv2.imread('uv_test.png')
result = image.copy()
tri2uv = (aux.verts_uvs*(size-1)).numpy().astype(np.int32)[faces.textures_idx.numpy()]
uv_texture = np.zeros((size, size,3))
width, height, channel = uv_texture.shape


for i in range(tri2uv.shape[0]):
    
    vertices = uv_color[i]['triangle']
    x1, y1 = vertices[0]
    x2, y2 = vertices[1]
    x3, y3 = vertices[2]
   
    for x, y in uv_color[i]['inside']:

        color1 = image[x1, y1]
        color2 = image[x2, y2]
        color3 = image[x3, y3]
        
        # 计算三角形的面积
        area = 0.5 * abs((x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)))

        # 计算三个顶点的颜色在三角形内的权重
        w1 = 0.5 * abs((x2 * (y3 - y) + x3 * (y - y2) + x * (y2 - y3))) / area
        w2 = 0.5 * abs((x3 * (y - y1) + x * (y1 - y3) + x1 * (y3 - y))) / area
        w3 = 0.5 * abs((x * (y1 - y2) + x1 * (y2 - y) + x2 * (y - y1))) / area

        # # 根据重心权重计算像素的颜色
        pixel_color = w1 * color1 + w2 * color2 + w3 * color3
        uv_texture[x, y] = pixel_color
        # print(w1, w2, w3)        

/tmp/ipykernel_9558/4184229157.py:26: RuntimeWarning: invalid value encountered in scalar divide
  w1 = 0.5 * abs((x2 * (y3 - y) + x3 * (y - y2) + x * (y2 - y3))) / area
/tmp/ipykernel_9558/4184229157.py:27: RuntimeWarning: invalid value encountered in scalar divide
  w2 = 0.5 * abs((x3 * (y - y1) + x * (y1 - y3) + x1 * (y3 - y))) / area
/tmp/ipykernel_9558/4184229157.py:28: RuntimeWarning: invalid value encountered in scalar divide
  w3 = 0.5 * abs((x * (y1 - y2) + x1 * (y2 - y) + x2 * (y - y1))) / area


In [10]:
cv2.imwrite('uv_final.png', uv_texture)

True

In [7]:
faces.textures_idx

tensor([[    0,     1,     2],
        [    3,     4,     5],
        [    6,     7,     8],
        ...,
        [32799, 32800, 32801],
        [32802, 32803, 32804],
        [32805, 32806, 32807]])

In [112]:
np.max(faces.verts_idx.numpy())

7305

In [79]:
verts

tensor([[ -48.1071,   67.3655, 1010.2392],
        [ -18.8887,   95.4126,  929.3578],
        [   7.3019,   80.2762,  900.0712],
        ...,
        [  65.3622,  150.5015, 1072.9222],
        [  52.5336,  151.1689, 1082.1195],
        [  38.6894,  153.1265, 1089.4698]])

In [137]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

# 创建示例数据
x = verts.numpy()[:, 0]
y = verts.numpy()[:, 1]
z = verts.numpy()[:, 2]

# 创建3D图形对象
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 绘制3D点
ax.scatter(x, y, z)

# 设置坐标轴标签
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# 显示图形
plt.show()

<IPython.core.display.Javascript object>

In [138]:
np.unique(faces.verts_idx.numpy().flatten())

array([   0,    1,    2, ..., 7303, 7304, 7305])

In [131]:
for i in np.unique(faces.verts_idx.numpy().flatten()):
    print(i)

0
1
2
3
4
5
6
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
83
84
86
87
88
89
90
91
92
93
94
95
96
97
98
99
101
103
104
105
106
107
108
109
110
111
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306

In [127]:

faces.textures_idx.numpy().flatten()

array([    0,     1,     2, ..., 32805, 32806, 32807])

In [82]:
faces.verts_idx

torch.Size([10936, 3])

In [81]:
faces.textures_idx

tensor([[    0,     1,     2],
        [    3,     4,     5],
        [    6,     7,     8],
        ...,
        [32799, 32800, 32801],
        [32802, 32803, 32804],
        [32805, 32806, 32807]])

In [31]:
aux

Properties(normals=None, verts_uvs=tensor([[0.3518, 0.4916],
        [0.3586, 0.5064],
        [0.3471, 0.5128],
        ...,
        [0.6662, 0.6382],
        [0.6761, 0.6513],
        [0.6772, 0.6545]]), material_colors=None, texture_images=None, texture_atlas=None)

In [59]:

import matplotlib.pyplot as plt
import numpy as np
import cv2

In [76]:
from pytorch3d.io import load_obj
verts, faces, aux = load_obj('../test_data/000220.obj')
uvs = aux.verts_uvs.numpy()
uvs[:, 0] = uvs[:, 0] * 1024
uvs[:, 1] = uvs[:, 1] * 1024

In [13]:
from pytorch3d.io import load_obj
import numpy as np
obj_paht = '/run/user/1000/gvfs/nfs:host=140.96.126.171,prefix=%2Fvolume1%2FS100/FY112_FRP/6674443_GHS/geom/tracked_mesh/E001_Neutral_Eyes_Open/000220.obj'
verts, faces, aux = load_obj(obj_paht)

An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/home/aaron/miniconda3/envs/3d/lib/python3.10/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/home/aaron/miniconda3/envs/3d/lib/python3.10/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt


In [14]:
transform = [0.9985507, 0.0019925737, -0.053781703, 82.27165,
-0.040035605, 0.695336, -0.7175688, 715.88446,
0.035966545, 0.71868205, 0.694408, 376.94455]
transform = np.array(transform).reshape((3, 4))
R = transform[:3, :3]
t = transform[:, 3]
verts = ((R@verts.numpy().T).T + t)


In [15]:
total_lines = 0
with open(
        obj_paht,
        'r') as f:
    total_lines = len(f.readlines())

txt = ''
with open(obj_paht,'r') as f:
    for i in range(verts.shape[0]):
        f.readline()
        txt += f'v {verts[i][0]} {verts[i][1]} {verts[i][2]}\n'
    for _ in range(verts.shape[0], total_lines):
        txt += f.readline()
# with open('../test_data/test.obj', 'w') as w:
with open(f'test.obj', 'w') as w:
    w.write(txt)

In [103]:
vert2uv = {}

for i, idxes in enumerate(faces.verts_idx.numpy()):
    for j, idx in enumerate(idxes):
        if idx not in vert2uv:
            vert2uv[idx] = aux.verts_uvs.numpy()[faces.textures_idx[i][j]]

In [108]:
len(vert2uv.keys())

5471

In [77]:
image = np.zeros((1024, 1024, 3), dtype=np.uint8)
uvs = uvs.astype(np.int32)
for i in uvs:
    print((i[0], i[1]))
    cv2.circle(image, (i[0], i[1]), 5, (0, 0, 255), -1)
cv2.imwrite('uv_test.png', image)

(360, 503)
(367, 518)
(355, 525)
(360, 503)
(355, 525)
(349, 509)
(372, 549)
(359, 555)
(350, 544)
(372, 549)
(350, 544)
(363, 538)
(404, 560)
(414, 569)
(404, 575)
(404, 560)
(404, 575)
(393, 567)
(417, 638)
(416, 640)
(404, 634)
(417, 638)
(404, 634)
(406, 632)
(372, 619)
(371, 615)
(384, 616)
(372, 619)
(384, 616)
(384, 620)
(360, 620)
(358, 616)
(371, 615)
(360, 620)
(371, 615)
(372, 619)
(405, 709)
(415, 704)
(417, 709)
(405, 709)
(417, 709)
(406, 715)
(466, 292)
(464, 288)
(479, 288)
(466, 292)
(479, 288)
(480, 291)
(466, 318)
(456, 322)
(452, 316)
(466, 318)
(452, 316)
(462, 311)
(440, 334)
(432, 340)
(428, 334)
(440, 334)
(428, 334)
(435, 328)
(389, 351)
(381, 349)
(384, 336)
(389, 351)
(384, 336)
(392, 339)
(388, 379)
(390, 394)
(383, 398)
(388, 379)
(383, 398)
(380, 380)
(417, 464)
(412, 473)
(404, 462)
(417, 464)
(404, 462)
(410, 454)
(425, 472)
(436, 478)
(432, 491)
(425, 472)
(432, 491)
(421, 483)
(512, 468)
(500, 469)
(501, 465)
(512, 468)
(501, 465)
(512, 465)
(270, 760)

True

In [17]:
import numpy as np


class AABB:
    def __init__(self, min_corner, max_corner):
        self.min_corner = min_corner
        self.max_corner = max_corner

    def intersect(self, ray_origin, ray_direction):
        epsilon = 1e-6  # 極小值
        t_min = (self.min_corner[0] - ray_origin[0]) / (ray_direction[0] + epsilon)
        t_max = (self.max_corner[0] - ray_origin[0]) / (ray_direction[0] + epsilon)

        if t_min > t_max:
            t_min, t_max = t_max, t_min

        ty_min = (self.min_corner[1] - ray_origin[1]) / (ray_direction[1] + epsilon)
        ty_max = (self.max_corner[1] - ray_origin[1]) / (ray_direction[1] + epsilon)

        if ty_min > ty_max:
            ty_min, ty_max = ty_max, ty_min

        if (t_min > ty_max) or (ty_min > t_max):
            return False

        if ty_min > t_min:
            t_min = ty_min

        if ty_max < t_max:
            t_max = ty_max

        tz_min = (self.min_corner[2] - ray_origin[2]) / (ray_direction[2] + epsilon)
        tz_max = (self.max_corner[2] - ray_origin[2]) / (ray_direction[2] + epsilon)

        if tz_min > tz_max:
            tz_min, tz_max = tz_max, tz_min

        if (t_min > tz_max) or (tz_min > t_max):
            return False

        return True


class KDNode:
    def __init__(self, objects, depth=0):
        self.left = None
        self.right = None
        self.aabb = None
        self.objects = objects
        self.depth = depth
        self.split_axis = depth % 3

        if len(objects) > 1:
            self.split()
        else:
            self.aabb = self.objects[0].aabb

    def split(self):
        sorted_objects = sorted(self.objects, key=lambda o: o.aabb.min_corner[self.split_axis])
        median_idx = len(sorted_objects) // 2
        
        self.left = KDNode(sorted_objects[:median_idx], self.depth + 1)
        self.right = KDNode(sorted_objects[median_idx:], self.depth + 1)
        self.aabb = AABB(self.left.aabb.min_corner, self.right.aabb.max_corner)


class KDTree:
    def __init__(self, objects):
        self.root = KDNode(objects)

    def intersect(self, ray_origin, ray_direction):
        intersections = []

        def traverse(node):
            if node.aabb.intersect(ray_origin, ray_direction):
                if node.left:
                    traverse(node.left)
                if node.right:
                    traverse(node.right)
                if not node.left and not node.right:
                    for obj in node.objects:
                        if obj.intersect(ray_origin, ray_direction):
                            intersections.append(obj)

        traverse(self.root)
        return intersections


class Triangle:
    def __init__(self, vertices):
        self.vertices = vertices
        self.aabb = AABB(np.min(vertices, axis=0), np.max(vertices, axis=0))

    def intersect(self, ray_origin, ray_direction):
        epsilon = 1e-6
        v0, v1, v2 = self.vertices

        edge1 = v1 - v0
        edge2 = v2 - v0
        pvec = np.cross(ray_direction, edge2)
        det = np.dot(edge1, pvec)

        if abs(det) < epsilon:
            return False

        inv_det = 1.0 / det
        tvec = ray_origin - v0
        u = np.dot(tvec, pvec) * inv_det

        if u < 0 or u > 1:
            return False

        qvec = np.cross(tvec, edge1)
        v = np.dot(ray_direction, qvec) * inv_det

        if v < 0 or u + v > 1:
            return False

        t = np.dot(edge2, qvec) * inv_det

        if t < 0:
            return False

        return True


# 创建场景对象
triangle1 = Triangle(np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]]))
triangle2 = Triangle(np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]]))
triangle3 = Triangle(np.array([[0, 0, 0], [0, 1, 0], [0, 0, 1]]))
objects = [triangle1, triangle2, triangle3]

# 创建 KD 树
kd_tree = KDTree(objects)

# 定义光线
ray_origin = np.array([0, 0, 0])
ray_direction = np.array([1, 0, 0])

# 光线追踪
intersections = kd_tree.intersect(ray_origin, ray_direction)

# 输出相交的物体
for obj in intersections:
    print(obj.vertices)



[[0 0 0]
 [0 0 1]
 [0 1 0]]
[[0 0 0]
 [0 1 0]
 [0 0 1]]


In [ ]:
import cv2
import mediapipe as mp

def main():
    # 初始化Mediapipe的人臉檢測模型
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection()

    # 讀取影片或攝影機串流
    cap = cv2.VideoCapture(0)  # 0代表預設攝影機

    while True:
        # 讀取影格
        ret, frame = cap.read()

        # 轉換影格至RGB格式
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 進行人臉檢測
        results = face_detection.process(rgb_frame)

        # 繪製檢測結果
        if results.detections:
            for detection in results.detections:
                # 取得檢測框的位置和置信度
                bbox = detection.location_data.relative_bounding_box
                x, y, w, h = int(bbox.xmin * frame.shape[1]), int(bbox.ymin * frame.shape[0]), \
                             int(bbox.width * frame.shape[1]), int(bbox.height * frame.shape[0])
                confidence = detection.score

                # 繪製檢測框和置信度
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f'Confidence: {confidence:.2f}', (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # 顯示影格
        cv2.imshow('Face Detection', frame)

        # 按下q鍵結束程式
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 釋放資源
    cap.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()

In [1]:
import cv2

In [14]:
import os
dirs = '/home/aaron/Desktop/multiface/6674443_GHS/images/E030_Open_Mouth_Wide_Tongue_Up_And_Back'
dest = '/home/aaron/Downloads/me/E030'


In [16]:
for view in os.listdir(dirs):
    for _, _, file_name in os.walk(f'{dirs}/{view}'):
        img = cv2.imread(f'{dirs}/{view}/{sorted(file_name)[0]}')
        cv2.imwrite(f'{dest}/{view}_{sorted(file_name)[0]}', img)
    

In [43]:
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh


class FaceMesh():

    def __init__(self, batch_size, kpt_num):

        self.face_mesh = mp_face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
        )
        self.kpt_num = kpt_num
        self.batch_size = batch_size
        self.error_idx = 0

        self.silhouette = [
            10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397,
            365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58,
            132, 93, 234, 127, 162, 21, 54, 103, 67, 109
        ]
        self.lipsUpperOuter = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
        self.lipsLowerOuter = [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
        self.lipsUpperInner = [
            78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308
        ]
        self.lipsLowerInner = [
            78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308
        ]

        self.rightEyeUpper0 = [246, 161, 160, 159, 158, 157, 173]
        self.rightEyeLower0 = [33, 7, 163, 144, 145, 153, 154, 155, 133]
        self.rightEyeUpper1 = [247, 30, 29, 27, 28, 56, 190]
        self.rightEyeLower1 = [130, 25, 110, 24, 23, 22, 26, 112, 243]
        self.rightEyeUpper2 = [113, 225, 224, 223, 222, 221, 189]
        self.rightEyeLower2 = [226, 31, 228, 229, 230, 231, 232, 233, 244]
        self.rightEyeLower3 = [143, 111, 117, 118, 119, 120, 121, 128, 245]

        self.rightEyebrowUpper = [156, 70, 63, 105, 66, 107, 55, 193]
        self.rightEyebrowLower = [35, 124, 46, 53, 52, 65]

        self.rightEyeIris = [473, 474, 475, 476, 477]

        self.leftEyeUpper0 = [466, 388, 387, 386, 385, 384, 398]
        self.leftEyeLower0 = [263, 249, 390, 373, 374, 380, 381, 382, 362]
        self.leftEyeUpper1 = [467, 260, 259, 257, 258, 286, 414]
        self.leftEyeLower1 = [359, 255, 339, 254, 253, 252, 256, 341, 463]
        self.leftEyeUpper2 = [342, 445, 444, 443, 442, 441, 413]
        self.leftEyeLower2 = [446, 261, 448, 449, 450, 451, 452, 453, 464]
        self.leftEyeLower3 = [372, 340, 346, 347, 348, 349, 350, 357, 465]

        self.leftEyebrowUpper = [383, 300, 293, 334, 296, 336, 285, 417]
        self.leftEyebrowLower = [265, 353, 276, 283, 282, 295]

        self.leftEyeIris = [468, 469, 470, 471, 472]

        self.midwayBetweenEyes = [168]

        self.noseTip = [1]
        self.noseBottom = [2]
        self.noseRightCorner = [98]
        self.noseLeftCorner = [327]

        self.rightCheek = [205]
        self.leftCheek = [425]

        self.mouth_idx = (self.lipsUpperOuter + self.lipsLowerOuter +
                          self.lipsUpperInner + self.lipsLowerInner)

        self.eyes_idx = (self.rightEyeUpper0 + self.rightEyeLower0 +
                         self.rightEyebrowUpper + self.rightEyebrowLower +
                         self.leftEyeUpper0 + self.leftEyeLower0 +
                         self.leftEyebrowUpper + self.leftEyebrowLower +
                         self.rightEyeIris + self.leftEyeIris)

        self.nose_idx = (self.noseTip + self.noseBottom +
                         self.noseRightCorner + self.noseLeftCorner)

        self.features_idx = (
            self.silhouette,
            self.mouth_idx,
            self.eyes_idx,
            self.nose_idx,
        )


    def detect(
        self,
        images,
        type = None,
        exp = None,
        camID = None,
    ):

        features = np.zeros(
            (self.batch_size,
             len(self.silhouette + self.mouth_idx + self.eyes_idx + self.nose_idx),
             2))

        for i, img in enumerate(images):
            # Convert the BGR image to RGB before processing.
            tmp_kpt = np.zeros((self.kpt_num, 2))
            results = self.face_mesh.process(img)

            # Only for debugging
            if not results.multi_face_landmarks:
                # print(f'{i}_fuck')
                # print(f'Image type: {type}')
                # print(f'EXP: {exp[i]}')
                # cv2.imwrite(
                #     f'err_{type}_{exp[i]}_{camID[i]}_{self.error_idx}.png',
                #     cv2.cvtColor(img, cv2.COLOR_RGB2BGR),
                # )
                # self.error_idx =self.error_idx % 10 + 1
                continue

            annotated_image = img.copy()
            for face_landmarks in results.multi_face_landmarks:

                for idx, data_point in enumerate(face_landmarks.landmark):
                    tmp_kpt[idx, 0] = data_point.x * annotated_image.shape[1]
                    tmp_kpt[idx, 1] = data_point.y * annotated_image.shape[0]
            
            start = 0
            for _, feat in enumerate(self.features_idx):
                features[i, start:start+len(feat)] = tmp_kpt[feat]
                start += len(feat)

        return features

In [49]:
fm = FaceMesh(1, 478)

In [55]:
img = cv2.imread('/home/aaron/Downloads/me/frame/frame_0009.png')
feature = fm.detect(np.expand_dims(img, 0))

In [56]:
for kpt in feature[0]:
    cv2.circle(img, (int(kpt[0]), int(kpt[1])), radius=4, color=(0, 0, 255), thickness=-1)

In [57]:
cv2.imwrite('testtsetsets.png', img)

True

In [8]:
import head_segmentation.segmentation_pipeline as seg_pipeline
import cv2

In [9]:
image = cv2.imread('/home/aaron/Downloads/me/IMG_5535.JPG')

In [10]:


segmentation_pipeline = seg_pipeline.HumanHeadSegmentationPipeline()

segmentation_map = segmentation_pipeline.predict(image)

2023-07-14 10:22:30.786 | WARNING  | head_segmentation.segmentation_pipeline:__init__:23 - Model /home/aaron/miniconda3/envs/3d/lib/python3.10/site-packages/head_segmentation/model/head_segmentation.ckpt doesn't exist. Downloading the model to /home/aaron/miniconda3/envs/3d/lib/python3.10/site-packages/head_segmentation/model/head_segmentation.ckpt.
Downloading...
From (uriginal): https://drive.google.com/uc?id=1Asm8JMFQwiHGGuQLXG_qnHvspWGdGuxw
From (redirected): https://drive.google.com/uc?id=1Asm8JMFQwiHGGuQLXG_qnHvspWGdGuxw&confirm=t&uuid=b9cc7e65-05f8-4ccb-a09e-27bfbc6e2c5a
To: /home/aaron/miniconda3/envs/3d/lib/python3.10/site-packages/head_segmentation/model/head_segmentation.ckpt
 13%|█▎        | 46.1M/360M [01:08<06:01, 867kB/s] 

KeyboardInterrupt: 

 13%|█▎        | 46.1M/360M [01:20<06:01, 867kB/s]

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import cv2
from pytorch3d.io import load_obj
verts, faces, aux = load_obj('/home/aaron/Desktop/multiface_version/MeshroomCache/Texturing/56b8ef30ab45ec16c2c40f92d8c67057084f9061/texturedMesh.obj')

An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/home/aaron/miniconda3/envs/3d/lib/python3.10/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/home/aaron/miniconda3/envs/3d/lib/python3.10/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt


In [3]:
verts.shape

torch.Size([162894, 3])

In [7]:
with open('../training_data/000220.bin', 'rb') as f:
    # 读取数据
    data = f.read()

# 解析数据
verts = np.frombuffer(data, dtype=np.float32)
verts = verts.reshape((7306, 3))
verts = torch.Tensor(verts.copy())

In [8]:
verts

tensor([[ -69.9181,  -23.5175,  -63.2600],
        [ -46.6825,  -75.1553,    7.2793],
        [ -23.4147,  -69.7849,   41.9569],
        ...,
        [  50.5686,  -83.1244, -138.4000],
        [  38.5625,  -80.9287, -148.4320],
        [  25.4284,  -80.5315, -157.1990]])

In [1]:
import numpy as np
import cv2

def rotation_matrix_to_euler_xyz(rot_matrix):
    # Convert the 3x3 rotation matrix to a 3x1 rotation vector
    rot_vector, _ = cv2.Rodrigues(rot_matrix)
    
    # Convert the rotation vector to Euler angles (XYZ order)
    roll_angle, pitch_angle, yaw_angle = rot_vector
    
    # Convert angles from radians to degrees
    roll_angle_deg = np.degrees(roll_angle)
    pitch_angle_deg = np.degrees(pitch_angle)
    yaw_angle_deg = np.degrees(yaw_angle)
    
    return roll_angle_deg, pitch_angle_deg, yaw_angle_deg

# Example usage:
rotation_matrix = np.array([[0.996236, 0.000852, 0.086679],
                            [0.027586, -0.95108, -0.307709],
                            [0.082176, 0.308941, -0.947524]])

roll, pitch, yaw = rotation_matrix_to_euler_xyz(rotation_matrix)
print("Roll angle (X-axis):", roll)
print("Pitch angle (Y-axis):", pitch)
print("Yaw angle (Z-axis):", yaw)

Roll angle (X-axis): [161.86737995]
Pitch angle (Y-axis): [1.18186956]
Yaw angle (Z-axis): [7.017519]


In [47]:
R = np.array([[ 0.65733950373107908, -0.25914178293195705, 0.70763713382879023],
[ 0.2795359293539032, 0.95587190317273873, 0.090380135678012383],
[ -0.69983172336985999, 0.13839957031643099, 0.70077180158836228]])

C = np.array([[0.36200976311954958],
[-0.067715034780622796],
[0.70427057366735701]])

In [146]:
import torch
import numpy as np
from pytorch3d.io import load_obj

R_o = [
                        "0.95248805003089743",
                        "0.13210295320050158",
                        "-0.27443637569397511",
                        "0.038004783274775203",
                        "0.84245316168469864",
                        "0.53742748982136401",
                        "0.30219555092121381",
                        "-0.52232315679391528",
                        "0.79740602510908021"
                    ]

C_o = [  
                        "0.74819188457856622",
                        "-1.4829816666376117",
                        "-0.17611059554482594"
    ]

In [147]:
obj_paht = '/media/aaron/work/ITRI_SSTC/S100/FY112_FRP/code/FRP/test_data/E01_KRTFREE_2021/MeshroomCache/Texturing/dd2a38dcf4e101fa33335c441913102d70d86693/texturedMesh.obj'
# obj_paht = 'MeshroomCache/Texturing/765aeed556c961561548c55866209afae8f59477/texturedMesh.obj'
verts, faces, aux = load_obj(obj_paht)

An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/home/aaron/miniconda3/envs/3d/lib/python3.10/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/home/aaron/miniconda3/envs/3d/lib/python3.10/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt


In [148]:
R = np.array([float(i) for i in R_o]).reshape((3,3), order='F')
C = np.array([float(i) for i in C_o]).reshape(3, 1)
t = -1 * R @ C


In [149]:

verts = torch.tensor(((R@verts.numpy().T).T + t.reshape(3, )), dtype=torch.float32)
verts = verts.numpy()

In [150]:
# verts[:, 1] *= -1
# verts[:, 2] *= -1

In [151]:
verts

array([[-0.03491604, -0.30957493,  2.2861001 ],
       [-0.08424645, -0.30147234,  2.2728865 ],
       [-0.00425541, -0.30225125,  2.277349  ],
       ...,
       [-0.1981709 , -0.11902177,  2.574281  ],
       [-0.1973439 , -0.11930052,  2.5771081 ],
       [-0.0974764 ,  0.17052397,  2.271439  ]], dtype=float32)

In [145]:
total_lines = 0
with open(
        obj_paht,
        'r') as f:
    total_lines = len(f.readlines())

txt = ''
with open(obj_paht,'r') as f:
    for i in range(verts.shape[0]):
        f.readline()
        txt += f'v {verts[i][0]} {verts[i][1]} {verts[i][2]}\n'
    for _ in range(verts.shape[0], total_lines):
        txt += f.readline()
# with open('../test_data/test.obj', 'w') as w:
with open(f'test.obj', 'w') as w:
    w.write(txt)

In [273]:
import torch
import numpy as np
from pytorch3d.io import load_obj

R_o = [
                        "0.99988281819799663",
                        "0.013847035881812944",
                        "0.0065275929499257011",
                        "-0.015078000889664664",
                        "0.81709507323402075",
                        "0.57630573065505919",
                        "0.002646462091813611",
                        "-0.5763366211633425",
                        "0.81720811018027628"
                    ]

C_o = [  
                        "-0.015169498866016628",
                        "-1.2359340183958105",
                        "-0.45485697953408488"
    ]

R = np.array([float(i) for i in R_o]).reshape((3,3), order='F')
C = np.array([float(i) for i in C_o]).reshape(3, 1)
t = -1 * R @ C

print(f'R: {R}')
print(f't: {t}')

R: [[ 0.99988282 -0.015078    0.00264646]
 [ 0.01384704  0.81709507 -0.57633662]
 [ 0.00652759  0.57630573  0.81720811]]
t: [[-0.00226393]
 [ 0.74793492]
 [ 1.08408769]]


In [274]:
import math
world_pos = np.array([-0.16820904612541199, -0.030312452465295792, 1.4954971075057983])
cam_pos = [0.48676346278241278, -0.022171128484136042, 0.08938904329579897]
cam_rotation_m = R

# first transform to cam_coordinate
# cam_coord = np.dot(cam_rotation_m.T, (np.array(world_pos) - np.array(cam_pos)))
cam_coord = (cam_rotation_m@ world_pos).T + cam_pos


img_w = 1334
img_h = 2048
# I guess the meshroom principle point is start from center so will shift it
principle_pt = [801.62599999999998, 997.66949999999997]
principle_pt = [principle_pt[0] + (img_w/2), principle_pt[1] + (img_h/2)]

focal_len = 26.5718
sensor_w = 4.6022999999999996
# someone said the focal length need to be in pixel instead of mm, so I also transform it
focal_len = focal_len * img_w / sensor_w

# then according to the equation
xh = cam_coord[0]/cam_coord[2]
yh = cam_coord[1]/cam_coord[2]
r2 = math.pow(xh, 2)*math.pow(yh, 2)
r4 = math.pow(r2, 2)
r6 = r2*r4

radial_mat = [0, 0, 0]
xhd = (1+(radial_mat[0]*r2)+(radial_mat[1]*r4)+(radial_mat[2]*r6)) * xh
yhd = (1+(radial_mat[0]*r2)+(radial_mat[1]*r4)+(radial_mat[2]*r6)) * yh

xd = -(focal_len*xhd) + principle_pt[0]
yd = -(focal_len*yhd) + principle_pt[1]


In [275]:
xd

-455.3799957844876

In [101]:
yd

2283.9565445480694

In [2]:
import json
with open('MeshroomCache/ConvertSfMFormat/35f1ad7d7036320c0e944d4325e4d2e18be70fdc/sfm.json', 'r') as json_file:
    data = json.load(json_file)


In [11]:
data['structure'][0].keys()

dict_keys(['landmarkId', 'descType', 'color', 'X', 'observations'])

In [13]:
data['structure'][0]['X']

['0.030472571030259132', '-0.2358691543340683', '2.0566079616546631']

In [2]:
import json

with open('/home/aaron/Desktop/test/MeshroomCache/StructureFromMotion/5fd79ba39718a5d917e3dbdff98eb99c2af25ea8/cameras.sfm', 'r') as f:
    data = json.load(f)


In [7]:
data.keys()

dict_keys(['version', 'featuresFolders', 'matchesFolders', 'views', 'intrinsics', 'poses'])

In [19]:
data['views']

[{'viewId': '127696682',
  'poseId': '127696682',
  'intrinsicId': '1458987727',
  'resectionId': '4',
  'path': '/home/aaron/Downloads/me/E01/400030_000220.png',
  'width': '1334',
  'height': '2048',
  'metadata': {'AliceVision:useWhiteBalance': '1'}},
 {'viewId': '135831270',
  'poseId': '135831270',
  'intrinsicId': '1458987727',
  'resectionId': '30',
  'path': '/home/aaron/Downloads/me/E01/400067_000220.png',
  'width': '1334',
  'height': '2048',
  'metadata': {'AliceVision:useWhiteBalance': '1'}},
 {'viewId': '166354825',
  'poseId': '166354825',
  'intrinsicId': '1458987727',
  'path': '/home/aaron/Downloads/me/E01/400055_000220.png',
  'width': '1334',
  'height': '2048',
  'metadata': {'AliceVision:useWhiteBalance': '1'}},
 {'viewId': '265957499',
  'poseId': '265957499',
  'intrinsicId': '1458987727',
  'resectionId': '25',
  'path': '/home/aaron/Downloads/me/E01/400061_000220.png',
  'width': '1334',
  'height': '2048',
  'metadata': {'AliceVision:useWhiteBalance': '1'}},


In [24]:
intrinsics = {intrin['intrinsicId']:intrin for intrin in data['intrinsics']}


In [20]:
data['intrinsics']

[{'intrinsicId': '1458987727',
  'width': '1334',
  'height': '2048',
  'sensorWidth': '-1',
  'sensorHeight': '-0.6513671875',
  'serialNumber': '/home/aaron/Downloads/me/E01',
  'type': 'radial3',
  'initializationMode': 'unknown',
  'pxInitialFocalLength': '-1',
  'pxFocalLength': '7941.1517964024642',
  'principalPoint': ['733.70000000000005', '979.00162256048793'],
  'distortionParams': ['0.021074569475600807',
   '-4.3246876312887874',
   '197.32842507424726'],
  'locked': '0'}]

In [33]:
import numpy as np
a = ['733.70000000000005', '979.00162256048793']
np.array()

TypeError: float() argument must be a string or a real number, not 'list'

In [29]:
poses = {pose['poseId']:pose for pose in data['poses']}

In [30]:
poses

{'127696682': {'poseId': '127696682',
  'pose': {'transform': {'rotation': ['0.97186655009111489',
     '0.04125566844639824',
     '-0.23189087656705118',
     '0.039086693459538475',
     '0.94263852087171385',
     '0.33151900301971776',
     '0.23226631096307224',
     '-0.33125607736285895',
     '0.91450629959655116'],
    'center': ['0.51516312062113012',
     '-1.0707711124033557',
     '-0.32563418237811198']},
   'locked': '0'}},
 '135831270': {'poseId': '135831270',
  'pose': {'transform': {'rotation': ['-0.6692198907318464',
     '0.09278288447223422',
     '-0.73724899063875815',
     '0.094981887758447806',
     '0.99471602180630536',
     '0.038967639903895977',
     '0.73696891308025825',
     '-0.043947381160316261',
     '-0.67449644094127925'],
    'center': ['1.8914755076582508',
     '-0.33014894829612818',
     '3.5888063606409988']},
   'locked': '0'}},
 '265957499': {'poseId': '265957499',
  'pose': {'transform': {'rotation': ['0.96281083538714574',
     '-0.172

In [34]:
a = {'a':1, 'b':2}

In [39]:
'b' in a.keys()

True

In [2]:
with open('../tmp/feature.txt', 'r') as d_data:
    content = d_data.read()

In [3]:
content

'[[19347, 0.3, 0.4],[19550, 0.3, 0.4],[19370, 0.3, 0.4],[19436, 0.3, 0.4],[12169, 0.3, 0.4],[12198, 0.3, 0.4],[28471, 0.3, 0.4],[18593, 0.3, 0.4],[18588, 0.3, 0.4],[26556, 0.3, 0.4],[26548, 0.3, 0.4],[19387, 0.3, 0.4],[19503, 0.3, 0.4],[19291, 0.3, 0.4],[19293, 0.3, 0.4],[13896, 0.3, 0.4],[13787, 0.3, 0.4],[14097, 0.3, 0.4],[26608, 0.3, 0.4],[26562, 0.3, 0.4],[26548, 0.3, 0.4],[19403, 0.3, 0.4],[19485, 0.3, 0.4],[19480, 0.3, 0.4],[19444, 0.3, 0.4],[19307, 0.3, 0.4],[13899, 0.3, 0.4],[13911, 0.3, 0.4],[14100, 0.3, 0.4],[18064, 0.3, 0.4],[14073, 0.3, 0.4],[26516, 0.3, 0.4],[19403, 0.3, 0.4],[19487, 0.3, 0.4],[19481, 0.3, 0.4],[19467, 0.3, 0.4],[13854, 0.3, 0.4],[13509, 0.3, 0.4],[13927, 0.3, 0.4],[14081, 0.3, 0.4],[13530, 0.3, 0.4],[14073, 0.3, 0.4],[26516, 0.3, 0.4],[24693, 0.3, 0.4],[24574, 0.3, 0.4],[24617, 0.3, 0.4],[26097, 0.3, 0.4],[26107, 0.3, 0.4],[26137, 0.3, 0.4],[12475, 0.3, 0.4],[24700, 0.3, 0.4],[12907, 0.3, 0.4],[12916, 0.3, 0.4],[12945, 0.3, 0.4],[26185, 0.3, 0.4],[26177, 

In [4]:
import ast

In [5]:
ast.literal_eval(content)

[[19347, 0.3, 0.4],
 [19550, 0.3, 0.4],
 [19370, 0.3, 0.4],
 [19436, 0.3, 0.4],
 [12169, 0.3, 0.4],
 [12198, 0.3, 0.4],
 [28471, 0.3, 0.4],
 [18593, 0.3, 0.4],
 [18588, 0.3, 0.4],
 [26556, 0.3, 0.4],
 [26548, 0.3, 0.4],
 [19387, 0.3, 0.4],
 [19503, 0.3, 0.4],
 [19291, 0.3, 0.4],
 [19293, 0.3, 0.4],
 [13896, 0.3, 0.4],
 [13787, 0.3, 0.4],
 [14097, 0.3, 0.4],
 [26608, 0.3, 0.4],
 [26562, 0.3, 0.4],
 [26548, 0.3, 0.4],
 [19403, 0.3, 0.4],
 [19485, 0.3, 0.4],
 [19480, 0.3, 0.4],
 [19444, 0.3, 0.4],
 [19307, 0.3, 0.4],
 [13899, 0.3, 0.4],
 [13911, 0.3, 0.4],
 [14100, 0.3, 0.4],
 [18064, 0.3, 0.4],
 [14073, 0.3, 0.4],
 [26516, 0.3, 0.4],
 [19403, 0.3, 0.4],
 [19487, 0.3, 0.4],
 [19481, 0.3, 0.4],
 [19467, 0.3, 0.4],
 [13854, 0.3, 0.4],
 [13509, 0.3, 0.4],
 [13927, 0.3, 0.4],
 [14081, 0.3, 0.4],
 [13530, 0.3, 0.4],
 [14073, 0.3, 0.4],
 [26516, 0.3, 0.4],
 [24693, 0.3, 0.4],
 [24574, 0.3, 0.4],
 [24617, 0.3, 0.4],
 [26097, 0.3, 0.4],
 [26107, 0.3, 0.4],
 [26137, 0.3, 0.4],
 [12475, 0.3, 0.4],
